# Genetic Algorithm for Neural Network Evolution
Testing and optimizing time-aware neural networks using genetic algorithms

In [ ]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import sys
import os
from dotenv import load_dotenv

load_dotenv()
ROOT = os.getenv('ROOT')
sys.path.append(ROOT)

from src.evolution import GeneticAlgorithm, create_simple_test_fitness, create_sphere_function_fitness, create_trading_fitness_function

## Test 1: Simple Optimization Problem
Validate genetic algorithm on sphere function

In [2]:
# Test genetic algorithm on sphere function
print("Testing Genetic Algorithm on Sphere Function")
print("=" * 50)

# Create genetic algorithm with small population for testing
ga_test = GeneticAlgorithm(
    population_size=20,
    crossover_rate=0.9,
    mutation_rate=0.1,
    mutation_strength=0.1,
    tournament_size=3,
    elitism_count=2
)

# Create sphere function fitness (minimization converted to maximization)
sphere_fitness = create_sphere_function_fitness()

# Run evolution for few generations
best_individual, evolution_stats = ga_test.run_evolution(
    fitness_function=sphere_fitness,
    generations=100,
    verbose=True
)

print(f"\nBest individual fitness: {sphere_fitness(best_individual):.6f}")
print(f"Best weights sum of squares: {np.sum(best_individual.get_weights()**2):.6f}")
print(f"Expected optimum: 0.0 (all weights should be near zero)")

Testing Genetic Algorithm on Sphere Function
🧬 Genetic Algorithm initialized (Population: 20)
🚀 Parallel processing: 16 workers, 1 threads each
🚀 Starting evolution (100 generations)...
⚠️ Parallel failed, using serial: Can't get local object 'create_sphere_function_fit...
Generation   0: Best=  0.2519, Mean=  0.1753, Std=0.0345 [0.1s]
⚠️ Parallel failed, using serial: Can't get local object 'create_sphere_function_fit...
⚠️ Parallel failed, using serial: Can't get local object 'create_sphere_function_fit...
⚠️ Parallel failed, using serial: Can't get local object 'create_sphere_function_fit...
⚠️ Parallel failed, using serial: Can't get local object 'create_sphere_function_fit...
⚠️ Parallel failed, using serial: Can't get local object 'create_sphere_function_fit...
⚠️ Parallel failed, using serial: Can't get local object 'create_sphere_function_fit...
⚠️ Parallel failed, using serial: Can't get local object 'create_sphere_function_fit...
⚠️ Parallel failed, using serial: Can't get lo

In [3]:
# Generate more realistic trading data with periodic patterns
n_points = 2000
base_times = np.arange(n_points)

# Create multiple periodic components of different frequencies
trend = 0.05 * base_times  # Slight upward trend
daily_cycle = 30 * np.sin(2 * np.pi * base_times / 100)  # Daily cycle
weekly_cycle = 50 * np.sin(2 * np.pi * base_times / 500)  # Weekly cycle  
monthly_cycle = 25 * np.sin(2 * np.pi * base_times / 1000)  # Monthly cycle

# Random walk component (realistic price movement)
random_walk = np.cumsum(np.random.normal(0, 8, n_points))

# Combine all components
base_price = 1000
prices = base_price + trend + daily_cycle + weekly_cycle + monthly_cycle + random_walk

# Add market volatility clusters (realistic feature)
volatility = 1 + 0.5 * np.abs(np.sin(2 * np.pi * base_times / 300))  # Volatility clustering
noise = np.random.normal(0, 3, n_points) * volatility
prices += noise

# Normalize to reasonable trading range (but don't clip - let it be dynamic)
price_range = np.max(prices) - np.min(prices)
target_range = 400  # Target range of $400
scale_factor = target_range / price_range
prices = base_price + (prices - np.mean(prices)) * scale_factor

# Create irregular timestamps
timestamps = []
price_data = []
current_time = 0.0

for i in range(n_points):
    rush_period = (i % 200) < 20  # More frequent rush periods
    time_increment = 0.1 if rush_period else 1.0
    
    current_time += time_increment
    timestamps.append(current_time)
    price_data.append(prices[i])

timestamps = np.array(timestamps)
price_data = np.array(price_data)

print(f"Generated {len(price_data)} price points")
print(f"Price range: {np.min(price_data):.2f} to {np.max(price_data):.2f}")
print(f"Price mean: {np.mean(price_data):.2f}, std: {np.std(price_data):.2f}")
print(f"Time range: {timestamps[0]:.1f} to {timestamps[-1]:.1f}")

# Visualize the price components
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Generated Price Series with Periodic Patterns', 'Price Components'),
    vertical_spacing=0.1
)

# Main price series
fig.add_trace(go.Scatter(
    x=base_times, y=price_data,
    mode='lines',
    name='Final Price Series',
    line=dict(color='blue', width=1)
), row=1, col=1)

# Show individual components
fig.add_trace(go.Scatter(
    x=base_times, y=base_price + daily_cycle,
    mode='lines',
    name='Daily Cycle',
    line=dict(color='red', width=1)
), row=2, col=1)

fig.add_trace(go.Scatter(
    x=base_times, y=base_price + weekly_cycle,
    mode='lines',
    name='Weekly Cycle', 
    line=dict(color='green', width=1)
), row=2, col=1)

fig.add_trace(go.Scatter(
    x=base_times, y=base_price + monthly_cycle,
    mode='lines',
    name='Monthly Cycle',
    line=dict(color='purple', width=1)
), row=2, col=1)

fig.update_layout(
    title="Realistic Price Data with Multiple Periodic Components",
    height=700,
    width=1200
)

fig.update_yaxes(title_text="Price", row=1, col=1)
fig.update_yaxes(title_text="Price Component", row=2, col=1)
fig.update_xaxes(title_text="Time", row=2, col=1)

fig.show()

Generated 2000 price points
Price range: 820.19 to 1220.19
Price mean: 1000.00, std: 111.03
Time range: 0.1 to 1820.0


## Generate Trading Data
Create price and timestamp data for neural network trading evolution

In [4]:
# Visualize price data
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=timestamps,
    y=price_data,
    mode='lines',
    name='Price Data',
    line=dict(color='blue', width=1)
))

fig.update_layout(
    title="Simulated Price Data",
    xaxis_title="Time",
    yaxis_title="Price",
    width=800,
    height=500
)

fig.show()


## Test 2: Trading Neural Network Evolution
Evolve neural networks for optimal trading performance

In [5]:
# Create trading fitness function
trading_fitness = create_trading_fitness_function(
    prices=price_data,
    timestamps=timestamps,
    risk_penalty=0.1,
    sharpe_weight=0.3
)

print("Testing Random Individual Performance")
print("=" * 40)

# Test a few random individuals for baseline
from src.evolution import Individual

random_scores = []
for i in range(10):
    individual = Individual()
    fitness = trading_fitness(individual)
    random_scores.append(fitness)
    print(f"Random individual {i+1}: {fitness:.4f}")

print(f"\nRandom baseline - Mean: {np.mean(random_scores):.4f}, Std: {np.std(random_scores):.4f}")
print(f"Best random: {np.max(random_scores):.4f}")

Testing Random Individual Performance


NameError: name 'normalized_prices' is not defined

In [ ]:
# Run genetic algorithm for trading neural network evolution
print("\nEvolution Trading Neural Networks")
print("=" * 40)

# Create genetic algorithm for trading
ga_trading = GeneticAlgorithm(
  population_size=30,
  crossover_rate=0.9,
  mutation_rate=0.05,  # Lower mutation for trading
  mutation_strength=0.1,
  tournament_size=3,
  elitism_count=3,
)

# Create trading fitness function
from src.evolution import create_trading_fitness_function
trading_fitness = create_trading_fitness_function(
  prices=price_data,
  timestamps=timestamps,
  risk_penalty=0.1,
  sharpe_weight=0.3
)

# Run evolution with direct fitness function
best_trader, trading_evolution_stats = ga_trading.run_evolution(
  fitness_function=trading_fitness,
  generations=50,
  verbose=True
)

# Test final fitness
final_fitness = trading_fitness(best_trader)

print(f"\nBest evolved individual fitness: {final_fitness:.6f}")
print(f"Improvement over random mean: {final_fitness -
np.mean(random_scores):.4f}")



Evolution Trading Neural Networks
🧬 Genetic Algorithm initialized (Population: 30)
🚀 Starting evolution (50 generations)...
Generation   0: Best=  2.6560, Mean=  0.0190, Std=0.8736 [1.3s]
Generation  10: Best=113.4838, Mean= 29.4408, Std=46.7958 [14.8s]


## Genetic Algorithm Results

The genetic algorithm successfully evolved neural networks for trading.

In [7]:
print("⚡ GENETIC ALGORITHM RESULTS")
print("=" * 40)

print(f"\n🧬 Genetic Algorithm:")
print(f"   • Population size: {ga_trading.population_size}")
print(f"   • Device: CPU (OPTIMIZED)")  # Fixed - no more population.individual_params
print(f"   • Generations: 50")

print(f"\n✨ Key Features:")
print(f"   • Ultra-optimized CPU-only neural networks")
print(f"   • Single-pass trading simulation")
print(f"   • Time-aware temporal memory")
print(f"   • Tournament selection with elitism")


⚡ GENETIC ALGORITHM RESULTS

🧬 Genetic Algorithm:
   • Population size: 30
   • Device: CPU (OPTIMIZED)
   • Generations: 50

✨ Key Features:
   • Ultra-optimized CPU-only neural networks
   • Single-pass trading simulation
   • Time-aware temporal memory
   • Tournament selection with elitism


In [8]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Visualize trading evolution progress
generations = list(range(len(trading_evolution_stats)))  # Generate generation numbers
best_fitness = [stat['best_fitness'] for stat in
trading_evolution_stats]
mean_fitness = [stat['mean_fitness'] for stat in
trading_evolution_stats]
std_fitness = [stat['std_fitness'] for stat in
trading_evolution_stats]

# Create matplotlib plot
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.plot(generations, best_fitness, 'g-', linewidth=2,
label='Best Fitness')
plt.plot(generations, mean_fitness, 'b-', alpha=0.7, label='Mean Fitness')
plt.fill_between(generations,
               np.array(mean_fitness) - np.array(std_fitness),
               np.array(mean_fitness) + np.array(std_fitness),
               alpha=0.3, color='blue', label='±1 Std Dev')
plt.xlabel('Generation')
plt.ylabel('Fitness')
plt.title('Trading Evolution Progress')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 2)
plt.plot(generations, std_fitness, 'orange', linewidth=2)
plt.xlabel('Generation')
plt.ylabel('Standard Deviation')
plt.title('Population Diversity')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Create interactive plotly visualization
fig = make_subplots(
  rows=2, cols=1,
  subplot_titles=('Fitness Evolution', 'Population Diversity'),
  vertical_spacing=0.1
)

# Fitness evolution
fig.add_trace(go.Scatter(
  x=generations, y=best_fitness,
  mode='lines+markers',
  name='Best Fitness',
  line=dict(color='green', width=2)
), row=1, col=1)

fig.add_trace(go.Scatter(
  x=generations, y=mean_fitness,
  mode='lines',
  name='Mean Fitness',
  line=dict(color='blue', width=1)
), row=1, col=1)

# Add random baseline
fig.add_hline(
  y=np.mean(random_scores),
  line_dash="dash",
  line_color="red",
  annotation_text="Random Baseline",
  row=1, col=1
)

# Population diversity (standard deviation)
fig.add_trace(go.Scatter(
  x=generations, y=std_fitness,
  mode='lines',
  name='Std Deviation',
  line=dict(color='orange', width=1)
), row=2, col=1)

fig.update_layout(
  title="Genetic Algorithm Evolution - Trading Neural Networks",
  height=700,
  width=1000
)

fig.update_xaxes(title_text="Generation", row=2, col=1)
fig.update_yaxes(title_text="Fitness", row=1, col=1)
fig.update_yaxes(title_text="Std Deviation", row=2, col=1)

fig.show()

print(f"📊 Evolution Summary:")
print(f"   • Starting fitness: {best_fitness[0]:.4f}")
print(f"   • Final fitness: {best_fitness[-1]:.4f}")
print(f"   • Improvement: {best_fitness[-1] -
best_fitness[0]:.4f}")
print(f"   • Best generation: {generations[np.argmax(best_fitness)]}")


NameError: name 'trading_evolution_stats' is not defined

## Analyze Best Evolved Individual
Test the performance of the evolved neural network

In [9]:
# Test best individual's trading performance
def detailed_trading_simulation(individual, prices, timestamps):
    individual.reset_state()
    portfolio_value = 1.0
    position = 0
    entry_price = 0.0
    
    actions = []
    executed_actions = []
    values = []
    
    for i, (price, timestamp) in enumerate(zip(prices, timestamps)):
        normalized_price = (price - 1000) / 100
        raw_action = individual.get_action(normalized_price, timestamp)
        
        # Position logic
        if position == 0 and raw_action == 2:
            position = 1
            entry_price = price
            executed_action = 2
        elif position == 1 and raw_action == 0:
            position = 0
            portfolio_value *= price / entry_price
            executed_action = 0
        else:
            executed_action = 1
        
        actions.append(raw_action)
        executed_actions.append(executed_action)
        values.append(portfolio_value)
    
    return np.array(actions), np.array(executed_actions), np.array(values)

# Simulate best individual
raw_actions, executed_actions, portfolio_values = detailed_trading_simulation(best_trader, price_data, timestamps)

print("Best Individual Performance Analysis")
print("=" * 40)
print(f"Final portfolio value: {portfolio_values[-1]:.4f}")
print(f"Total return: {(portfolio_values[-1] - 1.0) * 100:.2f}%")
print(f"Raw actions - Hold: {np.sum(raw_actions==1)}, Sell: {np.sum(raw_actions==0)}, Buy: {np.sum(raw_actions==2)}")
print(f"Executed actions - Hold: {np.sum(executed_actions==1)}, Sell: {np.sum(executed_actions==0)}, Buy: {np.sum(executed_actions==2)}")

# Count trades
num_buys = np.sum(executed_actions == 2)
num_sells = np.sum(executed_actions == 0)
print(f"Number of complete trades: {min(num_buys, num_sells)}")

NameError: name 'best_trader' is not defined

In [10]:
# Visualize best individual's trading performance
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Price Chart with Trading Signals', 'Portfolio Value Evolution'),
    vertical_spacing=0.1
)

# Price chart with signals
fig.add_trace(go.Scatter(
    x=timestamps, y=price_data,
    mode='lines',
    name='Price',
    line=dict(color='blue', width=1)
), row=1, col=1)

# Buy signals
buy_mask = executed_actions == 2
if np.any(buy_mask):
    fig.add_trace(go.Scatter(
        x=timestamps[buy_mask], y=price_data[buy_mask],
        mode='markers',
        name='Buy',
        marker=dict(color='green', symbol='triangle-up', size=8)
    ), row=1, col=1)

# Sell signals
sell_mask = executed_actions == 0
if np.any(sell_mask):
    fig.add_trace(go.Scatter(
        x=timestamps[sell_mask], y=price_data[sell_mask],
        mode='markers',
        name='Sell',
        marker=dict(color='red', symbol='triangle-down', size=8)
    ), row=1, col=1)

# Portfolio value
fig.add_trace(go.Scatter(
    x=timestamps, y=portfolio_values,
    mode='lines',
    name='Portfolio Value',
    line=dict(color='purple', width=2)
), row=2, col=1)

# Add horizontal line at 1.0 (break-even)
fig.add_hline(y=1.0, line_dash="dash", line_color="gray", row=2, col=1)

fig.update_layout(
    title="Best Evolved Individual - Trading Performance",
    height=800,
    width=1200
)

fig.update_yaxes(title_text="Price", row=1, col=1)
fig.update_yaxes(title_text="Portfolio Value", row=2, col=1)
fig.update_xaxes(title_text="Time", row=2, col=1)

fig.show()

NameError: name 'executed_actions' is not defined

## Compare Evolution vs Random
Statistical comparison of evolved vs random neural networks

In [11]:
# Generate larger sample of random individuals for comparison
print("Generating Random Population for Comparison")
random_population_scores = []
for i in range(100):
    individual = Individual()
    fitness = trading_fitness(individual)
    random_population_scores.append(fitness)

# Get final evolved population scores
evolved_scores = ga_trading.population.fitness_scores

print(f"\nStatistical Comparison:")
print(f"Random Population (n=100):")
print(f"  Mean: {np.mean(random_population_scores):.4f}")
print(f"  Std:  {np.std(random_population_scores):.4f}")
print(f"  Best: {np.max(random_population_scores):.4f}")
print(f"  Worst: {np.min(random_population_scores):.4f}")

print(f"\nEvolved Population (n={len(evolved_scores)}):")
print(f"  Mean: {np.mean(evolved_scores):.4f}")
print(f"  Std:  {np.std(evolved_scores):.4f}")
print(f"  Best: {np.max(evolved_scores):.4f}")
print(f"  Worst: {np.min(evolved_scores):.4f}")

improvement = np.mean(evolved_scores) - np.mean(random_population_scores)
print(f"\nMean Improvement: {improvement:.4f} ({improvement/np.mean(random_population_scores)*100:.1f}%)")

# Statistical significance test
from scipy import stats
t_stat, p_value = stats.ttest_ind(evolved_scores, random_population_scores)
print(f"T-test p-value: {p_value:.2e} ({'Significant' if p_value < 0.05 else 'Not significant'})")

Generating Random Population for Comparison


AttributeError: 'GeneticAlgorithm' object has no attribute 'population'

In [ ]:
# Visualize population comparison
fig = go.Figure()

# Random population histogram
fig.add_trace(go.Histogram(
    x=random_population_scores,
    name="Random Population",
    opacity=0.7,
    nbinsx=20,
    marker_color="red"
))

# Evolved population histogram
fig.add_trace(go.Histogram(
    x=evolved_scores,
    name="Evolved Population",
    opacity=0.7,
    nbinsx=20,
    marker_color="green"
))

# Add mean lines
fig.add_vline(
    x=np.mean(random_population_scores),
    line_dash="dash",
    line_color="red",
    annotation_text=f"Random Mean: {np.mean(random_population_scores):.3f}"
)

fig.add_vline(
    x=np.mean(evolved_scores),
    line_dash="dash",
    line_color="green",
    annotation_text=f"Evolved Mean: {np.mean(evolved_scores):.3f}"
)

fig.update_layout(
    title="Population Fitness Distribution: Random vs Evolved",
    xaxis_title="Fitness Score",
    yaxis_title="Frequency",
    barmode="overlay",
    width=1000,
    height=600
)

fig.show()

print(f"\n🎉 Genetic Algorithm Results Summary:")
print(f"✅ Successfully evolved neural networks for trading")
print(f"📈 Mean fitness improvement: {improvement:.4f}")
print(f"🏆 Best individual fitness: {np.max(evolved_scores):.4f}")
print(f"📊 Statistical significance: {p_value:.2e}")

In [6]:
from src.evolution import create_trading_fitness_function
trading_fitness = create_trading_fitness_function(
    prices=price_data,
    timestamps=timestamps,
    risk_penalty=0.1,
    sharpe_weight=0.3
)

ga_trading = GeneticAlgorithm(
    population_size=3,
    crossover_rate=0.9,
    mutation_rate=0.05,  # Lower mutation for trading
    mutation_strength=0.1,
    tournament_size=3,
    elitism_count=3,
)

🧬 Genetic Algorithm initialized (Population: 3)


In [8]:
import cProfile
from src.evolution import Individual

profiler = cProfile.Profile()
profiler.enable()

# Run genetic algorithm for trading neural network evolution
print("\nEvolution Trading Neural Networks")
print("=" * 40)

# Create genetic algorithm for trading
ga_trading = GeneticAlgorithm(
  population_size=30,
  crossover_rate=0.9,
  mutation_rate=0.05,  # Lower mutation for trading
  mutation_strength=0.1,
  tournament_size=3,
  elitism_count=3,
)

# Create trading fitness function
from src.evolution import create_trading_fitness_function
trading_fitness = create_trading_fitness_function(
  prices=price_data,
  timestamps=timestamps,
  risk_penalty=0.1,
  sharpe_weight=0.3
)

# Run evolution with direct fitness function
best_trader, trading_evolution_stats = ga_trading.run_evolution(
  fitness_function=trading_fitness,
  generations=5,
  verbose=True
)


profiler.disable()
stats = profiler.print_stats(sort='cumulative')



Evolution Trading Neural Networks
🧬 Genetic Algorithm initialized (Population: 30)
🚀 Starting evolution (5 generations)...
Generation   0: Best=  1.7434, Mean= -0.0404, Std=0.4285 [2.2s]
🏁 Evolution complete! Total time: 13.2s
   Final best fitness: 79.2030
   Avg time per generation: 2.63s
         29953526 function calls (28507611 primitive calls) in 13.159 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       12    0.000    0.000   13.155    1.096 base_events.py:1947(_run_once)
      7/3    0.000    0.000   13.153    4.384 interactiveshell.py:3636(run_code)
        1    0.000    0.000   13.153   13.153 genetic_algorithm.py:87(run_evolution)
      180    0.225    0.001   13.130    0.073 fitness.py:13(fitness_function)
   360000    0.590    0.000   12.862    0.000 individual.py:35(get_action_int)
        6    0.001    0.000   12.689    2.115 genetic_algorithm.py:30(evaluate_fitness)
   360000    0.207    0.000   10.821

In [ ]:
import cProfile
from src.evolution import Individual

profiler = cProfile.Profile()
profiler.enable()

best_trader, trading_evolution_stats = ga_trading.run_evolution(
    fitness_function=trading_fitness,
    generations=1,
    verbose=True
)

profiler.disable()
stats = profiler.print_stats(sort='cumulative')
